In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from multiprocessing import Pool
import matplotlib.pyplot as plt

In [3]:
import MiniTools as mini
import MicroTools as micro
import param_scan

In [14]:
# from MiniTools import fit

In [11]:
# --------------------------------------------------------------------------------
#                 Setting up Parameter Scan -- 2-dimensions
# --------------------------------------------------------------------------------
path_results = "fit_data/"
Npoints = 15

# Range of |U_{\mu4}|^2 scanned over 1e-4 to 1 in 30 steps
x_Vec = np.geomspace(1e-4, 0.9, Npoints)

# Range of gm4 for sterile decay model scanned over 0.1 to 100 in 30 steps
y_Vec = np.geomspace(1e-1, 1e2, Npoints)

# Cartesian product of grid
paramlist_decay = np.array(np.meshgrid(x_Vec, y_Vec)).T.reshape(-1, 2)

In [12]:
# Designed to run in parallel. Set the argument of "Pool" to 1 to disable this.
pool = Pool()
res = pool.map(param_scan.DecayReturnMicroBooNEChi2, paramlist_decay)
np.save(f"{path_results}/App_gm4_Um4sq_migrated_1_v2", res)